In [2]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from dataset import *
from wrapper import *
from utils import *


from transformers import AutoModelForSequenceClassification

In [17]:
train_df = pd.read_csv('../data/train.csv', sep='\t')

train_dataset_config = {
    'model_name':'hfl/chinese-macbert-base',
    'maxlength':64,
    'train_val_split':-1,
    'test':False, 
    'remove_username':False,
    'remove_punctuation':False, 
    'to_simplified':False, 
    'emoji_to_text':False, 
    'ner_config':'uer/roberta-base-finetuned-cluener2020-chinese', 
}

train = SimpleDataset(df=train_df, **train_dataset_config)
train.tokenize()
train.construct_dataset()

loading configuration file https://huggingface.co/hfl/chinese-macbert-base/resolve/main/config.json from cache at C:\Users\holaj/.cache\huggingface\transformers\dfbc5cae1d1506a97fa3e0014d34afdd480ea8cce63302d8df982725a3e5b77a.ddc6437c26ae4493a14d93f62bb62ae67d1e46c0de74ff95524ad468354a19f7
Model config BertConfig {
  "_name_or_path": "hfl/chinese-macbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform

In [23]:
model = AutoModelForSequenceClassification.from_pretrained('../sample_run/fold3/checkpoint-20')
model.cuda()

loading configuration file ../sample_run/fold3/checkpoint-20\config.json
Model config BertConfig {
  "_name_or_path": "../sample_run/fold3/checkpoint-20",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.21.0",
  "type_vocab_size": 2,
  "use_cache":

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [24]:
arguments = AdversarialTrainingArguments(
    output_dir='./eval', 
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16, 
    num_train_epochs=3,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",  # save checkpoint at each epoch
    learning_rate=1e-5, 
    load_best_model_at_end=True,
    epsilon=0, 
    alpha=.3, 
    gamma=.5, 
)

trainer = AdversarialTrainer(
    model=model, 
    args=arguments, 
    eval_dataset=train.dataset['train'], 
    tokenizer=train.tokenizer, 
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [25]:
prediction_output = trainer.predict(train.dataset['train'])

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ner_scores. If ner_scores are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 45248
  Batch size = 16


  0%|          | 0/2828 [00:00<?, ?it/s]

In [26]:
from copy import copy

eval_df = copy(train_df)
eval_df['prediction'] = np.argmax(prediction_output.predictions, 1)
eval_df = eval_df[['id', 'label', 'prediction', 'text']]

mislabelled_df = eval_df[eval_df['label'] != eval_df['prediction']]
mislabelled_df.to_csv('mislabelled-mini-with-ner.csv', encoding='utf-8')